# Hakyuu

1. Enter a number in each cell of the grid so that every region of size N contains all numbers from 1 to N exactly once.

2. If there are two cells with the same number Z in a row or column, there must be at least Z other cells between these two cells.

![](https://www.janko.at/Raetsel/Hakyuu/Regeln-01.gif) ![](https://www.janko.at/Raetsel/Hakyuu/Regeln-02.gif)


In [1]:
def readGrid(path):
    with open(f"../assets/data/Hakyuu/problems/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid1 = []
        areas1 = []
        for i in range(int(m)):
            grid1.append(f.readline())
        for i in range(int(m)):
            areas1.append(f.readline())
        
        grid = [g.strip().split(" ") for g in grid1]
        areas = [g.strip().split(" ") for g in areas1]
        return int(m), int(n), grid, areas

if __name__ == "__main__":
    m, n, grid, area = readGrid("394_10x10")
    for g in grid:
        print(g)
    for a in area:
        print(a)
    

['2', '-', '-', '-', '4', '-', '-', '-', '-', '-']
['-', '1', '-', '-', '6', '-', '-', '-', '-', '-']
['-', '-', '-', '-', '-', '-', '2', '-', '-', '-']
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
['-', '4', '-', '-', '-', '1', '4', '-', '-', '-']
['-', '-', '-', '6', '5', '-', '-', '-', '4', '-']
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
['-', '-', '-', '1', '-', '-', '-', '-', '-', '-']
['-', '-', '-', '-', '-', '5', '-', '-', '3', '-']
['-', '-', '-', '-', '-', '3', '-', '-', '-', '1']
['1', '1', '1', '1', '1', '2', '3', '4', '4', '5']
['6', '6', '7', '7', '8', '2', '2', '4', '4', '9']
['10', '6', '7', '7', '8', '11', '11', '4', '4', '9']
['6', '6', '6', '8', '8', '11', '12', '12', '12', '12']
['13', '14', '14', '8', '8', '11', '11', '15', '12', '12']
['16', '16', '14', '14', '14', '17', '18', '15', '15', '19']
['16', '16', '14', '20', '21', '17', '17', '15', '15', '19']
['22', '16', '22', '20', '21', '17', '17', '23', '24', '24']
['22', '22', '22', '20', '20', '20',

In [3]:
from ortools.sat.python import cp_model as cp 

def HakyuuSolver(X, Y, grid, area):
    model = cp.CpModel()
    solver = cp.CpSolver()

    x = dict()
    max_region_size = 0 # record the maximum region size
    region_dict = dict()
    for i in range(X):
        for j in range(Y):
            if area[i][j] in region_dict:
                region_dict[area[i][j]].append((i, j))
            else:
                region_dict[area[i][j]] = [(i, j)]
    
    for key, region in region_dict.items():
        region_size = len(region)
        max_region_size = max(max_region_size, region_size)
        for i, j in region:
            for k in range(1, region_size + 1):
                x[i, j, k] = model.NewBoolVar(f"x[{i},{j},{k}],r={key}")
            model.Add(sum([x[i, j, k] for k in range(1, region_size + 1)]) == 1)
            # Each with One 
        for k in range(1, region_size  + 1):
            model.Add(sum([x[i, j, k] for i, j in region]) == 1)
            # AllDifferent
    
    for i in range(X):
        for j in range(Y):
            if grid[i][j].isdigit():
                model.Add(x[i, j, int(grid[i][j])] == 1 )
                # Number equal for original board
            for k in range(1, max_region_size + 1):
                if (i, j, k) not in x:
                    break
                # iterate the cross position and set the constraint
                # 1. To right ... 
                temp_list = []
                tempK = 0
                while tempK <= k:
                    if j + tempK < Y and (i, j + tempK, k) in x:
                        temp_list.append(x[i, j + tempK, k])
                    elif j + tempK >= Y:
                        break 
                    tempK += 1
                model.Add(sum(temp_list) <= 1)
                
                # 2. To left ...
                temp_list = []
                tempK = 0
                while tempK <= k:
                    if j - tempK >= 0 and (i, j - tempK, k) in x:
                        temp_list.append(x[i, j - tempK, k])
                    elif j - tempK < 0:
                        break 
                    tempK += 1
                
                model.Add(sum(temp_list) <= 1)
                
                # 3. To below ...
                temp_list = []
                tempK = 0
                while tempK <= k:
                    if i + tempK < X and (i + tempK, j, k) in x:
                        temp_list.append(x[i + tempK, j, k])
                    elif i + tempK >= X:
                        break 
                    tempK += 1
                model.Add(sum(temp_list) <= 1)
                
                # 4. To above
                temp_list = []
                tempK = 0
                while tempK <= k:
                    if i - tempK >= 0 and (i - tempK, j, k) in x:
                        temp_list.append(x[i - tempK, j, k])
                    elif i - tempK < 0:
                        break 
                    tempK += 1
                model.Add(sum(temp_list) <= 1)
                
    status = solver.Solve(model)
    if status == cp.OPTIMAL:
        print("FOUND OPTIMAL")
        for i in range(X):
            for j in range(Y):
                for k in range(1, max_region_size + 1):
                    if (i, j, k) not in x:
                        break
                    elif solver.Value(x[i, j, k]) > 1e-3:
                        print(f"{k}", end = " ")
            print("")

if __name__ == "__main__":
    m, n, grid, area = readGrid("449_25x40")
    HakyuuSolver(m, n, grid, area)

FOUND OPTIMAL
2 1 5 2 4 1 6 3 5 4 2 1 3 1 4 1 5 1 2 1 4 2 5 6 7 2 4 1 2 1 3 1 2 1 4 3 1 6 8 4 
1 3 1 6 8 2 7 4 1 3 6 2 1 5 2 3 4 8 6 5 3 1 2 1 3 1 8 2 1 4 5 6 3 2 1 4 7 1 2 3 
4 1 3 7 2 4 9 6 2 8 5 1 2 4 3 7 2 1 3 2 6 9 3 2 5 4 2 3 7 1 2 4 1 6 2 5 3 4 1 2 
2 5 4 2 1 3 2 1 4 7 1 5 6 1 9 4 1 2 5 1 10 6 7 8 4 3 1 5 1 3 6 7 2 3 1 2 4 3 5 6 
1 4 2 1 3 6 1 7 3 5 2 1 4 2 6 5 3 1 8 4 2 5 1 7 2 1 3 8 2 5 1 2 1 4 3 6 5 1 2 1 
3 2 1 8 2 1 4 5 7 3 9 6 2 3 5 1 10 3 2 7 4 8 2 5 6 2 4 9 3 2 8 5 3 1 4 1 2 5 6 4 
2 1 6 1 7 2 3 4 5 2 1 3 1 7 4 2 8 5 3 1 9 4 1 10 3 5 7 4 6 1 3 9 4 5 2 7 8 2 4 3 
1 3 5 2 4 1 6 3 2 1 4 2 7 9 2 1 6 4 1 2 1 3 4 2 1 3 5 1 2 3 4 2 1 10 5 2 9 3 1 8 
6 2 1 3 1 5 2 9 1 4 3 1 2 6 8 3 1 2 4 3 2 7 9 1 5 6 1 2 1 4 1 3 2 9 8 5 3 6 10 7 
3 1 7 4 5 3 8 1 6 2 7 9 4 5 3 1 2 6 7 8 12 10 3 11 2 1 3 1 4 9 2 1 5 3 7 1 6 2 1 5 
1 5 2 1 6 4 1 2 3 5 8 1 6 4 7 5 3 9 1 2 4 1 6 2 3 4 1 5 8 2 3 6 9 1 4 2 7 8 2 6 
2 4 9 5 8 1 7 6 4 1 2 3 5 8 1 2 4 7 2 5 11 2 1 3 4 10 2 6 3 7 1 8 1 4 2 6 1 3 4 1 
4 8 